In [ ]:
import FinanceDataReader as fdr
import pandas as pd

def get_base_stock_list():
    kospi = fdr.StockListing('KOSPI')
    kosdaq = fdr.StockListing('KOSDAQ')
    df = pd.concat([kospi, kosdaq], ignore_index=True)
    print(df.columns)

    # 보통주 필터링
    df = df[~df['Name'].str.contains('우$|우B$|우선|리츠|스팩', regex=True)]
    df = df[['Code', 'Name', 'Market']]
    df.columns = ['code', 'name', 'market']
    return df.reset_index(drop=True)

# 실행 예시
base_stock_list = get_base_stock_list()
base_stock_list.head()
print(f"✅ base_stock_list 종목 수: {len(base_stock_list)}")


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

# 압축 해제된 xml 경로
xml_path = os.path.expanduser("~/Desktop/경종설/CORPCODE.xml")

tree = ET.parse(xml_path)
root = tree.getroot()

corp_list = []
for child in root:
    corp_code = child.findtext("corp_code")
    corp_name = child.findtext("corp_name")
    stock_code = child.findtext("stock_code")
    
    # ✅ 비어 있지 않은 stock_code만 수집
    if stock_code and stock_code.strip() != '':
        corp_list.append({
            "corp_code": corp_code,
            "corp_name": corp_name.strip(),
            "code": stock_code.strip()
        })

corp_df = pd.DataFrame(corp_list)
corp_df = corp_df[['code', 'corp_name', 'corp_code']]
corp_df.head()
print(f"✅ corp_df 종목 수: {len(corp_df)}")

# 병합: 종목코드를 기준으로 FDR 리스트에 DART의 corp_code 추가
merged_df = pd.merge(base_stock_list, corp_df, on='code', how='left')

# 병합 결과 확인
print("🔍 병합된 종목 수:", len(merged_df))
print("❗ corp_code가 없는 종목 수:", merged_df['corp_code'].isna().sum())
print("✅ 병합 예시:")
print(merged_df.head())

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

# ✅ 본인의 DART API 인증키 입력
api_key = "bcf1a97ba847da6fc613bce32bd1ef18301e03d3"

# ✅ 상장일 수집 함수
def get_listing_date(corp_code):
    url = f"https://opendart.fss.or.kr/api/company.json?crtfc_key={api_key}&corp_code={corp_code}"
    try:
        res = requests.get(url).json()
        est_dt = res.get("est_dt")
        if est_dt:
            return pd.to_datetime(est_dt, format="%Y%m%d")
    except:
        return pd.NaT
    return pd.NaT

# ✅ 전체 종목 대상 상장일 수집 (tqdm으로 진행 상황 표시)
merged_df["listing_date"] = [
    get_listing_date(corp_code) if pd.notnull(corp_code) else pd.NaT 
    for corp_code in tqdm(merged_df["corp_code"])
]

# ✅ 결과 확인
merged_df[['code', 'name', 'market', 'listing_date']].head()


In [ ]:
import os
import time
import requests
import pandas as pd
from tqdm import tqdm

# ✅ DART API KEY 설정
api_key = "5fe747154efe9d0028aa5f0f989d0e79c32f4b20".strip()

# ✅ 수집할 항목 정의
ACCOUNT_NAMES = {
    "자본총계": "자본총계",
    "매출액": "매출액",
    "수익": "수익",  
    "당기순이익": "당기순이익",
    "현금배당총액": "현금배당총액"
}

# ✅ DART 재무제표 요청 함수
def get_dart_facts(corp_code, year, report_code):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
    params = {
        "crtfc_key": api_key,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": report_code,
        "fs_div": "CFS"  # 연결재무제표 기준
    }
    try:
        res = requests.get(url, params=params).json()
        if res.get("status") != "000":
            return {}
        data = res.get("list", [])
        results = {}
        for row in data:
            account = row["account_nm"]
            if account in ACCOUNT_NAMES.values():
                # 중복 방지 (예: 매출과 수익 모두 있을 때)
                if account == "수익" and "매출액" in results:
                    continue
                results[account] = row["thstrm_amount"]
        return results
    except:
        return {}

# ✅ 연도 설정
target_years = [2020, 2021, 2022, 2023, 2024]
report_code = "11011"  # 1분기 사업보고서

# ✅ Google Drive 경로 설정
base_path = "/content/drive/MyDrive/경종설/연도별데이터"

# ✅ 연도별 수집
for year in target_years:
    print(f"\n📦 {year}년 재무지표 수집 시작")

    file_path = os.path.join(base_path, f"{year}_보통주_리스트.csv")
    df = pd.read_csv(file_path)

    result_rows = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"{year} 수집 중"):
        corp_code = str(row["corp_code"]).zfill(8)
        code = str(row["code"]).zfill(6)
        name = row["name"]

        facts = get_dart_facts(corp_code, str(year), report_code)
        result = {
            "종목코드": code,
            "기업명": name,
            "자본총계": facts.get("자본총계"),
            "매출액": facts.get("매출액") or facts.get("수익"),
            "당기순이익": facts.get("당기순이익"),
            "현금배당총액": facts.get("현금배당총액"),
        }
        result_rows.append(result)
        time.sleep(0.05)  # API 제한 완화

    # ✅ 저장
    out_df = pd.DataFrame(result_rows)
    out_file = os.path.join(base_path, f"{year}_원시재무지표.csv")
    out_df.to_csv(out_file, index=False)
    print(f"✅ {year} 저장 완료 → {out_file}")

In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import os
from tqdm import tqdm
import time

# Google Drive 경로
base_path = "/content/drive/MyDrive/경종설/연도별데이터"

# 발행주식수 가져오기 (KOSPI + KOSDAQ)
stocks_info = pd.concat([
    fdr.StockListing('KOSPI'),
    fdr.StockListing('KOSDAQ')
], ignore_index=True)
stocks_info['Code'] = stocks_info['Code'].apply(lambda x: str(x).zfill(6))

# 연도별 반복
target_years = [2020, 2021, 2022, 2023, 2024]

for year in target_years:
    print(f"\n📦 {year}년 시가총액 계산 및 병합 시작")

    # 원시재무지표 파일 불러오기
    file_path = os.path.join(base_path, f"{year}_원시재무지표.csv")
    df = pd.read_csv(file_path)

    marcap_list = []

    # 종목별 시총 계산
    for code in tqdm(df['종목코드'], desc=f"{year}"):
        code = str(code).zfill(6)

        try:
            # 종가 수집 (연초 기준일)
            stock_data = fdr.DataReader(code, f"{year}-01-02", f"{year}-01-02")
            if stock_data.empty:
                marcap_list.append(None)
                continue

            close_price = stock_data['Close'].values[0]

            # 발행주식수
            stock_info = stocks_info[stocks_info['Code'] == code]
            if stock_info.empty or pd.isna(stock_info['Stocks'].values[0]):
                marcap_list.append(None)
                continue

            shares = stock_info['Stocks'].values[0]

            # 시가총액 계산
            marcap = close_price * shares
            marcap_list.append(marcap)

        except:
            marcap_list.append(None)

        time.sleep(0.02)  # 과부하 방지용

    # 시가총액 칼럼 추가 (현금배당총액은 두고 새로 추가)
    df['시가총액'] = marcap_list

    # 저장
    out_file = os.path.join(base_path, f"{year}_원시재무지표_시가총액포함.csv")
    df.to_csv(out_file, index=False)

    print(f"✅ {year}년 저장 완료 → {out_file}")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import os

# ✅ 배당수익률 크롤링 함수 (네이버 기준)
def get_dividend_yield(code):
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')

        per_table = soup.find('table', class_='per_table')
        for row in per_table.find_all('tr'):
            th_elements = row.find_all('th')
            td_elements = row.find_all('td')

            for th, td in zip(th_elements, td_elements):
                if '배당수익률' in th.text:
                    value = td.text.strip().replace('%','').replace(',','')
                    if value != '-' and value != '':
                        return float(value)
        return None
    except:
        return None

# ✅ 로컬 경로 설정 (VSCode 기준)
base_path = "/Users/gun/Desktop/경종설/연도별데이터"
target_years = [2022, 2023, 2024]

# ✅ 연도별 배당수익률 덮어쓰기
for year in target_years:
    print(f"\n📦 {year}년 배당수익률 덮어쓰기 시작")

    file_path = os.path.join(base_path, f"{year}_원시재무지표_시가총액포함.csv")
    df = pd.read_csv(file_path)

    div_yield_list = []
    for code in tqdm(df['종목코드'], desc=f"{year} 배당수익률 수집"):
        div_yield = get_dividend_yield(str(code).zfill(6))
        div_yield_list.append(div_yield)
        time.sleep(0.2)  # 네이버 차단 방지

    # 칼럼명 변경 ('현금배당총액' → '배당수익률')
    if '현금배당총액' in df.columns:
        df = df.rename(columns={'현금배당총액': '배당수익률'})
    else:
        if '배당수익률' not in df.columns:
            df['배당수익률'] = None

    # 값 덮어쓰기
    df['배당수익률'] = div_yield_list

    # 기존 파일 덮어쓰기 저장
    df.to_csv(file_path, index=False)
    print(f"✅ {year}년 배당수익률 덮어쓰기 완료 → {file_path}")

In [ ]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from datetime import timedelta
from tqdm import tqdm

# ✅ 기본 경로 및 설정
base_path = "/Users/gun/Desktop/경종설/연도별데이터"
years = [2020, 2021, 2022, 2023, 2024]

# ✅ FDR 발행주식수 수집
stocks_info = pd.concat([fdr.StockListing('KOSPI'), fdr.StockListing('KOSDAQ')], ignore_index=True)
stocks_info['Code'] = stocks_info['Code'].astype(str).str.zfill(6)

# ✅ EPS dict (딕셔너리로 선언)
eps_dict_all = {}
final_results = []

# ✅ 모멘텀/수급지표 계산 함수
def get_momentum_factors(code, year):
    try:
        base_date = pd.to_datetime(f"{year}-01-01")
        for i in range(10):
            data = fdr.DataReader(code, base_date + timedelta(days=i), base_date + timedelta(days=i))
            if not data.empty:
                base_date = data.index[0]
                break
        else:
            return None

        data = fdr.DataReader(code, base_date - pd.DateOffset(days=370), base_date)
        if data.empty:
            return None

        def calc_return(delta_days):
            ref_date = base_date - timedelta(days=delta_days)
            past_data = data[data.index <= ref_date]
            if past_data.empty:
                return None
            return (data['Close'].iloc[-1] - past_data['Close'].iloc[-1]) / past_data['Close'].iloc[-1]

        ret_1m = calc_return(30)
        ret_3m = calc_return(90)
        ret_6m = calc_return(180)
        ret_12m = calc_return(365)

        vol_3m = data[data.index >= (base_date - timedelta(days=90))]['Close'].pct_change().std() * np.sqrt(252)
        vol_12m = data['Close'].pct_change().std() * np.sqrt(252)

        latest_volume = data['Volume'].iloc[-1]
        avg_volume_20d = data['Volume'][-20:].mean()
        volume_growth = (latest_volume - avg_volume_20d) / avg_volume_20d if avg_volume_20d != 0 else None

        shares = stocks_info[stocks_info['Code'] == code]['Stocks'].values
        shares = shares[0] if len(shares) > 0 else None
        turnover = latest_volume / shares if shares else None

        return {
            '종목코드': code,
            '연도': year,
            '1개월수익률': ret_1m,
            '3개월수익률': ret_3m,
            '6개월수익률': ret_6m,
            '12개월수익률': ret_12m,
            '3개월변동성': vol_3m,
            '12개월변동성': vol_12m,
            '최근거래량': latest_volume,
            '평균거래량증가율': volume_growth,
            '거래회전율': turnover
        }
    except:
        return None

# ✅ 연도별 루프
for year in years:
    print(f"\n📦 {year}년 데이터 처리중...")

    # ✅ 원시재무지표 로드
    file_path = f"{base_path}/{year}_원시재무지표_시가총액포함.csv"
    df_financial = pd.read_csv(file_path)

    # ✅ 종목코드 통일 및 연도 부여
    df_financial['종목코드'] = df_financial['종목코드'].astype(str).str.zfill(6)
    df_financial['연도'] = year

    # ✅ 발행주식수 병합
    df_financial = df_financial.merge(stocks_info[['Code', 'Stocks']], left_on='종목코드', right_on='Code', how='left')
    df_financial = df_financial.rename(columns={'Stocks': '발행주식수'}).drop(columns=['Code'])

    # ✅ 숫자형 변환
    for col in ['자본총계', '매출액', '당기순이익', '시가총액', '발행주식수']:
        df_financial[col] = pd.to_numeric(df_financial[col].astype(str).str.replace(',', '').replace('nan', '').replace('', '0'), errors='coerce')

    # ✅ 지표 계산
    df_financial['EPS'] = df_financial['당기순이익'] / df_financial['발행주식수']
    df_financial['순이익률'] = df_financial['당기순이익'] / df_financial['매출액']
    df_financial['PER'] = df_financial['시가총액'] / df_financial['당기순이익']
    df_financial['PBR'] = df_financial['시가총액'] / df_financial['자본총계']
    df_financial['PSR'] = df_financial['시가총액'] / df_financial['매출액']
    df_financial['ROE'] = df_financial['당기순이익'] / df_financial['자본총계']

    # ✅ EPS 성장률 계산
    eps_growth_list = []
    for idx, row in df_financial.iterrows():
        code = row['종목코드']
        current_eps = row['EPS']
        prev_eps = eps_dict_all.get((code, year - 1), None)
        eps_growth_list.append((current_eps - prev_eps) / prev_eps if prev_eps and prev_eps != 0 else None)

    df_financial['EPS성장률'] = eps_growth_list

    # ✅ EPS dict 업데이트
    eps_dict_all.update(df_financial.set_index(['종목코드', '연도'])['EPS'].to_dict())

    # ✅ 모멘텀/수급지표 tqdm 직렬 처리
    momentum_results = []
    for code in tqdm(df_financial['종목코드'], desc=f"{year}년 모멘텀/수급 계산중"):
        res = get_momentum_factors(code, year)
        if res:
            momentum_results.append(res)

    df_momentum = pd.DataFrame(momentum_results)

    # ✅ 병합 및 결과 저장
    df_final = pd.merge(df_financial, df_momentum, on=['종목코드', '연도'], how='left')
    final_results.append(df_final)

# ✅ 전체 연도 통합 및 파일 저장
df_total = pd.concat(final_results, ignore_index=True)
output_path = f"{base_path}/2020_2024_지표통합_완성본.csv"
df_total.to_csv(output_path, index=False)
print(f"✅ 최종 파일 저장 완료: {output_path}")

In [ ]:
import pandas as pd

# ✅ 경로 설정
base_path = "/Users/gun/Desktop/경종설/연도별데이터"
years = [2020, 2021, 2022, 2023, 2024]

# ✅ 전체 통합본 불러오기 (방금 구한 결과)
df_total = pd.read_csv(f"{base_path}/2020_2024_지표통합_완성본.csv")

# ✅ 덧붙일 컬럼 목록 (구한 지표들)
new_columns = [
    '연도', '발행주식수', 'EPS', '순이익률', 'PER', 'PBR', 'PSR', 'ROE', 'EPS성장률',
    '1개월수익률', '3개월수익률', '6개월수익률', '12개월수익률',
    '3개월변동성', '12개월변동성', '최근거래량', '평균거래량증가율', '거래회전율'
]

# ✅ 연도별 반복 병합
for year in years:
    print(f"📦 {year}년 병합 진행중...")

    # 연도별 원시재무지표 파일 불러오기
    file_path = f"{base_path}/{year}_원시재무지표_시가총액포함.csv"
    df_base = pd.read_csv(file_path)

    # 종목코드 형식 맞추기
    df_base['종목코드'] = df_base['종목코드'].astype(str).str.zfill(6)

   # 덧붙일 데이터 필터링 (해당 연도)
    df_year = df_total[df_total['연도'] == year][['종목코드'] + new_columns]

    # 종목코드 형식 통일
    df_year['종목코드'] = df_year['종목코드'].astype(str).str.zfill(6)

    # 중복된 연도 컬럼 제거 (원본 파일에 연도 있으면)
    if '연도' in df_base.columns:
        df_year = df_year.drop(columns=['연도'])

    # 병합
    df_merged = pd.merge(df_base, df_year, on='종목코드', how='left')

    # 결과 저장 (덮어쓰기 또는 다른 이름)
    output_path = f"{base_path}/{year}_원시재무지표_지표추가.csv"
    df_merged.to_csv(output_path, index=False)

    print(f"✅ {year}년 파일 저장 완료: {output_path}")

In [ ]:
import pandas as pd
import requests
import zipfile
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm

# ✅ DART API 인증키
api_key = 'bcf1a97ba847da6fc613bce32bd1ef18301e03d3'
base_path = "/Users/gun/Desktop/경종설/연도별데이터"

# ✅ corp_code 다운로드 (쿼터 초과시 백업 사용)
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
r = requests.get(url)

if b'PK' in r.content[:10]:
    with open(f"{base_path}/corpCode.zip", "wb") as f:
        f.write(r.content)

    with zipfile.ZipFile(f"{base_path}/corpCode.zip", "r") as zip_ref:
        zip_ref.extractall(base_path)

    tree = ET.parse(f"{base_path}/CORPCODE.xml")
    root = tree.getroot()

    data = []
    for child in root:
        corp_code = child.find('corp_code').text
        stock_code = child.find('stock_code').text
        if stock_code.strip():
            data.append({'종목코드': stock_code.zfill(6), 'corp_code': corp_code})

    df_corp_code = pd.DataFrame(data)
    print(f"✅ 최신 corp_code 매핑 완료 ({len(df_corp_code)}개)")
else:
    print("❌ corp_code 다운로드 실패 (쿼터 초과)")
    df_corp_code = pd.read_csv(f"{base_path}/backup_corp_code_mapping.csv")

# ✅ DART 재무지표 수집 함수
def get_dart_financial(corp_code, year, report_code='11011'):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bsns_year': year,
        'reprt_code': report_code
    }
    r = requests.get(url, params=params)
    data = r.json()

    result = {'자산총계': None, '부채총계': None, '영업이익': None}
    if data.get('status') == '000' and 'list' in data:
        cfs_data = [item for item in data['list'] if item['fs_div'] == 'CFS']
        ofs_data = [item for item in data['list'] if item['fs_div'] == 'OFS']

        def extract_value(account_nm, target_data):
            for item in target_data:
                amount = item['thstrm_amount']
                if item['account_nm'].strip() == account_nm and amount and amount != '-':
                    return int(amount.replace(',', ''))
            return None

        for account in ['자산총계', '부채총계', '영업이익']:
            result[account] = extract_value(account, cfs_data) or extract_value(account, ofs_data)

    return result

# ✅ 연도별 루프 (2020~2024)
years = [2020, 2021, 2022, 2023, 2024]

for year in years:
    print(f"\n📊 {year}년 DART 지표 병합 및 계산 중...")

    # 원시재무지표 파일에서 '당기순이익' 가져오기
    df_base = pd.read_csv(f"{base_path}/{year}_원시재무지표_시가총액포함.csv")
    df_base['종목코드'] = df_base['종목코드'].astype(str).str.zfill(6)

    if '당기순이익' not in df_base.columns:
        raise KeyError(f"❌ {year}_원시재무지표_시가총액포함.csv 에 '당기순이익' 컬럼이 없습니다.")
    else:
        print(f"✅ {year}년 '당기순이익' 컬럼 정상 확인 ({df_base['당기순이익'].notna().sum()}개 값 존재)")

    # 지표추가 파일 불러오기
    df_financial = pd.read_csv(f"{base_path}/{year}_원시재무지표_지표추가.csv")
    df_financial['종목코드'] = df_financial['종목코드'].astype(str).str.zfill(6)

    # corp_code 병합
    df_financial = pd.merge(df_financial, df_corp_code, on='종목코드', how='left')

    # ✅ 당기순이익 병합 (suffix 충돌 방지)
    df_financial = pd.merge(
        df_financial,
        df_base[['종목코드', '당기순이익']],
        on='종목코드',
        how='left',
        suffixes=('', '_from_base')
    )

    # ✅ 당기순이익 컬럼명 정리
    if '당기순이익_from_base' in df_financial.columns:
        df_financial['당기순이익'] = df_financial['당기순이익_from_base']
        df_financial.drop(columns=['당기순이익_from_base'], inplace=True)
    elif '당기순이익' not in df_financial.columns:
        raise KeyError(f"❌ '당기순이익' 병합 실패 (df_financial 컬럼 확인 필요)")

    # ✅ DART 재무지표 수집
    dart_data = []
    for _, row in tqdm(df_financial.iterrows(), total=len(df_financial), desc=f"{year} DART 수집"):
        if pd.isna(row['corp_code']):
            dart_data.append({'자산총계': None, '부채총계': None, '영업이익': None})
        else:
            result = get_dart_financial(row['corp_code'], year)
            dart_data.append(result)

    df_dart = pd.DataFrame(dart_data)
    df_financial = pd.concat([df_financial, df_dart], axis=1)

    # ✅ 숫자형 변환
    for col in ['당기순이익', '자산총계', '부채총계', '영업이익']:
        if col in df_financial.columns:
            df_financial[col] = pd.to_numeric(df_financial[col], errors='coerce')

    # ✅ 최종 지표 계산
    df_financial['ROA'] = df_financial['당기순이익'] / df_financial['자산총계']
    df_financial['부채비율'] = df_financial['부채총계'] / (df_financial['자산총계'] - df_financial['부채총계'])
    df_financial['영업이익률'] = df_financial['영업이익'] / df_financial['자산총계']

    # ✅ 저장 (corp_code 제거 후)
    df_financial.drop(columns=['corp_code'], inplace=True)
    df_financial.to_csv(f"{base_path}/{year}_원시재무지표_지표추가_DART완성.csv", index=False)

    print(f"✅ {year}년 최종 파일 저장 완료: {base_path}/{year}_원시재무지표_지표추가_DART완성.csv")

In [ ]:
import pandas as pd
from tqdm import tqdm

base_path = "/Users/gun/Desktop/경종설/연도별데이터"
output_path = "/Users/gun/Desktop/경종설"

years = [2020, 2021, 2022, 2023, 2024]

for year in years:
    print(f"\n📦 {year}년 최종 데이터셋 병합 진행...")

    # ✅ 기본 원시재무지표 + 지표추가 파일 불러오기
    df_base = pd.read_csv(f"{base_path}/{year}_원시재무지표_지표추가.csv")
    df_base['종목코드'] = df_base['종목코드'].astype(str).str.zfill(6)

    # ✅ DART완성 파일 불러오기 (자산총계, 부채총계, 영업이익, ROA 등 포함)
    df_dart = pd.read_csv(f"{base_path}/{year}_원시재무지표_지표추가_DART완성.csv")
    df_dart['종목코드'] = df_dart['종목코드'].astype(str).str.zfill(6)

    # ✅ EPS성장률 계산을 위한 EPS 딕셔너리 준비
    if year > 2020:
        df_prev = pd.read_csv(f"{base_path}/{year-1}_원시재무지표_지표추가_DART완성.csv")
        df_prev['종목코드'] = df_prev['종목코드'].astype(str).str.zfill(6)
        eps_prev_dict = df_prev.set_index('종목코드')['EPS'].to_dict()
    else:
        eps_prev_dict = {}

    # ✅ EPS성장률 계산
    eps_growth_list = []
    for idx, row in df_dart.iterrows():
        code = row['종목코드']
        current_eps = row.get('EPS', None)
        prev_eps = eps_prev_dict.get(code, None)

        if pd.notna(current_eps) and pd.notna(prev_eps) and prev_eps != 0:
            eps_growth = (current_eps - prev_eps) / prev_eps
        else:
            eps_growth = None

        eps_growth_list.append(eps_growth)

    df_dart['EPS성장률'] = eps_growth_list

    # ✅ ROA (재계산 안전망)
    if '당기순이익' in df_dart.columns and '자산총계' in df_dart.columns:
        df_dart['ROA'] = df_dart['당기순이익'] / df_dart['자산총계']

    # ✅ 병합할 컬럼만 필터링 (중복 제거용)
    dart_columns = ['종목코드', '자산총계', '부채총계', '영업이익', 'ROA', '부채비율', '영업이익률', 'EPS성장률']
    df_dart = df_dart[dart_columns]

    # ✅ 병합 수행 (종목코드 기준)
    df_merged = pd.merge(df_base, df_dart, on='종목코드', how='left')

    # ✅ 결과 저장 (연도별 데이터셋 파일로)
    save_file = f"{output_path}/{year}_데이터셋.csv"
    df_merged.to_csv(save_file, index=False)

    print(f"✅ {year}년 데이터셋 완성: {save_file}")

In [79]:
import pandas as pd

# ✅ 경로 설정
base_path = "/Users/gun/Desktop/경종설"
years = [2020, 2021, 2022, 2023, 2024]

for year in years:
    print(f"\n📦 {year}년 당기순이익 덮어쓰기 중...")

    # 원본 파일 (당기순이익 값이 있는 파일)
    file_시가총액포함 = f"{base_path}/연도별데이터/{year}_원시재무지표_시가총액포함.csv"
    df_source = pd.read_csv(file_시가총액포함)
    df_source['종목코드'] = df_source['종목코드'].astype(str).str.zfill(6)

    # 덮어쓸 파일 (데이터셋)
    file_데이터셋 = f"{base_path}/{year}_데이터셋.csv"
    df_target = pd.read_csv(file_데이터셋)
    df_target['종목코드'] = df_target['종목코드'].astype(str).str.zfill(6)

    # 당기순이익 덮어쓰기 (종목코드 기준)
    df_target = pd.merge(
        df_target.drop(columns=['당기순이익'], errors='ignore'), 
        df_source[['종목코드', '당기순이익']], 
        on='종목코드', 
        how='left'
    )

    # ✅ 결과 저장 (덮어쓰기)
    df_target.to_csv(file_데이터셋, index=False)

    print(f"✅ {year}년 당기순이익 덮어쓰기 완료: {file_데이터셋}")


📦 2020년 당기순이익 덮어쓰기 중...
✅ 2020년 당기순이익 덮어쓰기 완료: /Users/gun/Desktop/경종설/2020_데이터셋.csv

📦 2021년 당기순이익 덮어쓰기 중...
✅ 2021년 당기순이익 덮어쓰기 완료: /Users/gun/Desktop/경종설/2021_데이터셋.csv

📦 2022년 당기순이익 덮어쓰기 중...
✅ 2022년 당기순이익 덮어쓰기 완료: /Users/gun/Desktop/경종설/2022_데이터셋.csv

📦 2023년 당기순이익 덮어쓰기 중...
✅ 2023년 당기순이익 덮어쓰기 완료: /Users/gun/Desktop/경종설/2023_데이터셋.csv

📦 2024년 당기순이익 덮어쓰기 중...
✅ 2024년 당기순이익 덮어쓰기 완료: /Users/gun/Desktop/경종설/2024_데이터셋.csv


In [81]:
import pandas as pd

# ✅ 경로 및 연도 설정
base_path = "/Users/gun/Desktop/경종설"
years = [2020, 2021, 2022, 2023, 2024]

for year in years:
    print(f"\n📊 {year}년 ROA 계산 중...")

    file_path = f"{base_path}/{year}_데이터셋.csv"
    df = pd.read_csv(file_path)

    # ✅ 종목코드 포맷 보정
    df['종목코드'] = df['종목코드'].astype(str).str.zfill(6)

    # ✅ 문자열(,) 제거 후 숫자형 변환
    df['당기순이익'] = df['당기순이익'].astype(str).str.replace(',', '').astype(float)
    df['자산총계'] = df['자산총계'].astype(str).str.replace(',', '').astype(float)

    # ✅ ROA 계산 (당기순이익 / 자산총계 * 100)
    df['ROA'] = (df['당기순이익'] / df['자산총계']) * 100

    # ✅ 결과 저장
    df.to_csv(file_path, index=False)

    print(f"✅ {year}년 ROA 계산 완료: {file_path}")


📊 2020년 ROA 계산 중...
✅ 2020년 ROA 계산 완료: /Users/gun/Desktop/경종설/2020_데이터셋.csv

📊 2021년 ROA 계산 중...
✅ 2021년 ROA 계산 완료: /Users/gun/Desktop/경종설/2021_데이터셋.csv

📊 2022년 ROA 계산 중...
✅ 2022년 ROA 계산 완료: /Users/gun/Desktop/경종설/2022_데이터셋.csv

📊 2023년 ROA 계산 중...
✅ 2023년 ROA 계산 완료: /Users/gun/Desktop/경종설/2023_데이터셋.csv

📊 2024년 ROA 계산 중...
✅ 2024년 ROA 계산 완료: /Users/gun/Desktop/경종설/2024_데이터셋.csv


In [84]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# ✅ 파일 경로
base_path = "/Users/gun/Desktop/경종설"
data_file = f"{base_path}/2020_데이터셋.csv"

# ✅ 2020 데이터셋 불러오기
df_2020 = pd.read_csv(data_file)
df_2020['종목코드'] = df_2020['종목코드'].astype(str).str.zfill(6)

# ✅ 네이버 금융에서 2019년 당기순이익 가져오는 함수
def get_net_income_2019(code):
    url = f"https://finance.naver.com/item/main.nhn?code={code}"
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
        table = soup.select('div.section.cop_analysis div.sub_section tbody')[0]
        rows = table.find_all('tr')

        for row in rows:
            if '당기순이익' in row.get_text():
                values = [td.get_text().strip().replace(',', '') for td in row.find_all('td')]
                if len(values) >= 2 and values[1] != '':
                    return int(values[1])  # 두 번째 값이 2019년
        return None
    except Exception as e:
        return None

# ✅ EPS 성장률 계산
eps_growth_list = []
for idx, row in tqdm(df_2020.iterrows(), total=len(df_2020), desc="2019 EPS 크롤링 중"):
    code = row['종목코드']
    issued_shares = row['발행주식수']
    eps_2020 = row['EPS']

    net_income_2019 = get_net_income_2019(code)

    if net_income_2019 is not None and issued_shares != 0:
        eps_2019 = net_income_2019 / issued_shares
        if eps_2019 != 0:
            eps_growth = (eps_2020 - eps_2019) / eps_2019
        else:
            eps_growth = None
    else:
        eps_growth = None

    eps_growth_list.append(eps_growth)

# ✅ 반영 후 저장
df_2020['EPS성장률'] = eps_growth_list
df_2020.to_csv(data_file, index=False)

print(f"✅ 2020년 EPS 성장률 네이버 기준 계산 완료: {data_file}")

2019 EPS 크롤링 중: 100%|██████████| 2477/2477 [08:38<00:00,  4.78it/s]


✅ 2020년 EPS 성장률 네이버 기준 계산 완료: /Users/gun/Desktop/경종설/2020_데이터셋.csv


In [88]:
import pandas as pd
import os

# ✅ 경로 설정
base_path = "/Users/gun/Desktop/경종설"
years = [2020, 2021, 2022, 2023, 2024]

# ✅ 전체 데이터셋 통합
final_df_list = []

for year in years:
    file_path = f"{base_path}/{year}_데이터셋.csv"
    df = pd.read_csv(file_path)
    df['연도'] = year  # 혹시 빠져있으면 연도컬럼 보강
    final_df_list.append(df)

# ✅ 전체 통합
df_final = pd.concat(final_df_list, ignore_index=True)

# ✅ 결과 저장
output_path = f"{base_path}/2020_2024_데이터셋_최종통합.csv"
df_final.to_csv(output_path, index=False)

print(f"✅ 최종 통합 파일 저장 완료: {output_path}")

✅ 최종 통합 파일 저장 완료: /Users/gun/Desktop/경종설/2020_2024_데이터셋_최종통합.csv


In [3]:
import pandas as pd
import os

# ✅ 파일 경로 설정 (경로 수정)
base_path = '/Users/gun/Desktop/경종설'  # 👉 여기 본인 경로로 수정
file_names = [
    '2020_데이터셋.csv',
    '2021_데이터셋.csv',
    '2022_데이터셋.csv',
    '2023_데이터셋.csv',
    '2024_데이터셋.csv',
    '2020_2024_데이터셋_최종통합.csv'
]

# ✅ 포맷 정리 함수 (결측치는 그대로 유지)
def clean_dataset_strict(df):
    # 문자형 컬럼 공백 제거
    str_cols = df.select_dtypes(include=['object']).columns
    for col in str_cols:
        df[col] = df[col].astype(str).str.strip()
    
    # 종목코드 6자리 통일
    if '종목코드' in df.columns:
        df['종목코드'] = df['종목코드'].astype(str).str.zfill(6)
    
    # 수치형 변환 (오류값만 NaN 처리)
    for col in df.columns:
        if col not in str_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

# ✅ 파일별 덮어쓰기 처리
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    df = pd.read_csv(file_path)
    df_cleaned = clean_dataset_strict(df)
    df_cleaned.to_csv(file_path, index=False)  # 덮어쓰기 저장
    print(f"🔄 {file_name} 포맷 정리 완료")

🔄 2020_데이터셋.csv 포맷 정리 완료
🔄 2021_데이터셋.csv 포맷 정리 완료
🔄 2022_데이터셋.csv 포맷 정리 완료
🔄 2023_데이터셋.csv 포맷 정리 완료
🔄 2024_데이터셋.csv 포맷 정리 완료
🔄 2020_2024_데이터셋_최종통합.csv 포맷 정리 완료


In [6]:
import pandas as pd
import numpy as np
import os

# ✅ 파일 경로 설정 (수정)
base_path = '/Users/gun/Desktop/경종설/Raw_dataset'  # 👉 본인 경로로 수정
file_names = [
    '2020_데이터셋.csv',
    '2021_데이터셋.csv',
    '2022_데이터셋.csv',
    '2023_데이터셋.csv',
    '2024_데이터셋.csv',
    '2020_2024_데이터셋_최종통합.csv'
]

# ✅ 클리핑 + Z-score 함수
def clip_and_standardize(df, columns, lower_percentile=0.01, upper_percentile=0.99):
    df_result = df.copy()
    
    for col in columns:
        if col in df_result.columns:
            # 클리핑 (Winsorizing)
            lower = df_result[col].quantile(lower_percentile)
            upper = df_result[col].quantile(upper_percentile)
            df_result[col] = np.clip(df_result[col], lower, upper)

            # Z-score 표준화 (결측치는 그대로)
            mean = df_result[col].mean()
            std = df_result[col].std()
            df_result[col] = df_result[col].apply(lambda x: (x - mean) / std if pd.notnull(x) else x)
    
    return df_result

# ✅ 스코어링 대상 지표 목록
score_columns = [
    'PER', 'PBR', 'PSR', '배당수익률',     # 밸류지표
    'ROE', 'ROA', 'EPS성장률', '영업이익률', # 퀄리티지표
    '수익률', '평균거래량증가율'             # 모멘텀지표
]

# ✅ 원본 유지, 스코어링 파일 새로 저장
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    df = pd.read_csv(file_path)
    df_clipped = clip_and_standardize(df, score_columns)

    # 새 파일명: ex) 2020_데이터셋_스코어링.csv
    file_name_no_ext = os.path.splitext(file_name)[0]
    output_file = f"{file_name_no_ext}_스코어링.csv"
    output_path = os.path.join(base_path, output_file)

    df_clipped.to_csv(output_path, index=False)
    print(f"✅ {output_file} 클리핑 & z-score 저장 완료")

✅ 2020_데이터셋_스코어링.csv 클리핑 & z-score 저장 완료
✅ 2021_데이터셋_스코어링.csv 클리핑 & z-score 저장 완료
✅ 2022_데이터셋_스코어링.csv 클리핑 & z-score 저장 완료
✅ 2023_데이터셋_스코어링.csv 클리핑 & z-score 저장 완료
✅ 2024_데이터셋_스코어링.csv 클리핑 & z-score 저장 완료
✅ 2020_2024_데이터셋_최종통합_스코어링.csv 클리핑 & z-score 저장 완료


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from joblib import Parallel, delayed
from tqdm import tqdm
import FinanceDataReader as fdr

# ✅ 전체 CSV 로딩
df = pd.read_csv("2020_2024_리밸런싱_데이터셋_최종.csv")
df["기준월"] = pd.to_datetime(df["기준월"])
df["종목코드"] = df["종목코드"].astype(str).str.zfill(6)

# ✅ 수집 대상 컬럼
market_cols = [
    "1개월수익률", "3개월수익률", "6개월수익률", "12개월수익률",
    "3개월변동성", "12개월변동성", "최근거래량", "평균거래량증가율"
]

# ✅ 수집 함수 정의
def get_market_indicators(row):
    try:
        code = row["종목코드"]
        기준월 = row["기준월"]
        start_date = (기준월 - pd.DateOffset(months=13)).strftime("%Y-%m-%d")
        end_date = (기준월 + timedelta(days=1)).strftime("%Y-%m-%d")
        df_price = fdr.DataReader(code, start=start_date, end=end_date)
        if df_price.empty:
            return [None]*8

        df_price = df_price.sort_index()
        기준일 = 기준월 if 기준월 in df_price.index else df_price.index[df_price.index.get_loc(기준월, method='pad')]

        def calc_return(end, m):
            s = end - pd.DateOffset(months=m)
            w = df_price.loc[s:end]
            return (w["Close"].iloc[-1] / w["Close"].iloc[0]) - 1 if len(w) >= 2 else None

        def calc_vol(end, m):
            s = end - pd.DateOffset(months=m)
            return df_price.loc[s:end]["Close"].pct_change().std()

        return [
            calc_return(기준일, 1),
            calc_return(기준일, 3),
            calc_return(기준일, 6),
            calc_return(기준일, 12),
            calc_vol(기준일, 3),
            calc_vol(기준일, 12),
            df_price.loc[기준일]["Volume"],
            df_price["Volume"].iloc[-20:].mean() / df_price["Volume"].iloc[-60:].mean() - 1
            if len(df_price) >= 60 else None
        ]
    except:
        return [None]*8

# ✅ 병렬 수집 + 진행률
results = Parallel(n_jobs=-1)(
    delayed(get_market_indicators)(row) for row in tqdm(df.to_dict("records"))
)

# ✅ 결과 덮어쓰기
df[market_cols] = pd.DataFrame(results, columns=market_cols)
df.to_csv("2020_2024_시장지표_업데이트_완료.csv", index=False)
print("✅ 저장 완료: 2020_2024_시장지표_업데이트_완료.csv")

In [1]:
import pandas as pd

# ✅ 파일 경로
file_path = '/Users/gun/Desktop/경종설/2020_2024_리밸런싱_지표_22개.csv'

# ✅ CSV 파일 로드
df = pd.read_csv(file_path)

# ✅ 확인할 컬럼 목록
target_cols = [
    '1개월수익률', '3개월수익률', '6개월수익률', '12개월수익률',
    '3개월변동성', '12개월변동성', '평균거래량증가율', '거래회전율', '시가총액'
]

# ✅ 컬럼별 결측치 개수 출력
missing_counts = df[target_cols].isnull().sum()
print("✅ 시장지표별 결측치 개수:")
print(missing_counts)

✅ 시장지표별 결측치 개수:
1개월수익률       199
3개월수익률       199
6개월수익률       199
12개월수익률      199
3개월변동성       234
12개월변동성      234
평균거래량증가율    2900
거래회전율       3747
시가총액         199
dtype: int64


In [ ]:
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import numpy as np

# ✅ 파일 경로 및 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2020_2024_리밸런싱_지표_22개.csv'
df = pd.read_csv(file_path, dtype={'종목코드': str})
df['기준월'] = pd.to_datetime(df['기준월'])

# ✅ 결측치가 있는 항목만 필터링
target_cols = ['3개월변동성', '12개월변동성', '평균거래량증가율', '거래회전율']
df_missing = df[df[target_cols].isnull().any(axis=1)].copy()

# ✅ 종목코드를 yfinance 형식으로 변경
def to_ticker(code):
    return code + '.KS' if code.startswith(('0', '1')) else code + '.KQ'

# ✅ 결측치 계산 함수
def compute_missing(row):
    code = row['종목코드']
    ticker = to_ticker(code)
    기준월 = row['기준월']
    
    try:
        end = 기준월
        start = end - pd.DateOffset(months=13)
        data = yf.download(ticker, start=start.strftime('%Y-%m-%d'), end=end.strftime('%Y-%m-%d'), progress=False)

        if data.empty or len(data) < 60:
            return row

        data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
        prev = data['Volume_MA'].iloc[-60:-30].mean()
        recent = data['Volume_MA'].iloc[-30:].mean()
        row['평균거래량증가율'] = (recent - prev) / prev if prev != 0 else np.nan

        data['Return'] = data['Close'].pct_change()
        row['3개월변동성'] = data['Return'].iloc[-60:-30].std()
        row['12개월변동성'] = data['Return'].std()

        market_cap = row['시가총액'] if '시가총액' in row and not pd.isna(row['시가총액']) else None
        if market_cap:
            turnover = data['Volume'].iloc[-20:].mean() * 20 / market_cap
            row['거래회전율'] = turnover

    except Exception as e:
        print(f"❌ {code} 오류: {e}")
    
    return row

# ✅ 병렬 처리 실행
print("🚀 결측치 보완 중...")
results = []
row_list = [row for _, row in df_missing.iterrows()]
with ThreadPoolExecutor(max_workers=10) as executor:
    for updated_row in tqdm(executor.map(compute_missing, row_list), total=len(row_list)):
        results.append(updated_row)

# ✅ 업데이트 반영
df_updated = pd.DataFrame(results)
df_combined = pd.concat([df.drop(df_missing.index), df_updated], ignore_index=True)

# ✅ 저장
save_path = file_path.replace('.csv', '_결측치보완.csv')
df_combined.to_csv(save_path, index=False)
print(f"✅ 저장 완료: {save_path}")

In [17]:
import pandas as pd

# ✅ 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/2020_2024_리밸런싱_지표_22개_결측치보완.csv'

# ✅ CSV 파일 로드
df = pd.read_csv(file_path)

# ✅ 확인할 컬럼 목록
target_cols = [
    '1개월수익률', '3개월수익률', '6개월수익률', '12개월수익률',
    '3개월변동성', '12개월변동성', '평균거래량증가율', '거래회전율', '시가총액'
]

# ✅ 컬럼별 결측치 개수 출력
missing_counts = df[target_cols].isnull().sum()
print("✅ 시장지표별 결측치 개수:")
print(missing_counts)

✅ 시장지표별 결측치 개수:
1개월수익률       199
3개월수익률       199
6개월수익률       199
12개월수익률      199
3개월변동성        76
12개월변동성       76
평균거래량증가율    2636
거래회전율       2242
시가총액         199
dtype: int64


e69b5ce728486584ac3ae69bd5fdc5101b85c54b

In [6]:
import pandas as pd

# 🔑 파일 경로
REBAL_PATH = "/Users/gun/Desktop/경종설/국내/2020_2024_리밸런싱_지표_22개.csv"
DART_CORP_MAP_PATH = "/Users/gun/Desktop/경종설/국내/DART/corp_code_mapping.csv"
SAVE_MATCHED_PATH = "/Users/gun/Desktop/경종설/국내/DART/보유종목_dart코드매핑.csv"

# ✅ 1. 내가 보유한 종목 불러오기
rebal_df = pd.read_csv(REBAL_PATH)
rebal_df['기업명'] = rebal_df['기업명'].astype(str).str.strip()
unique_names = rebal_df['기업명'].unique()

# ✅ 2. DART 상장기업 매핑 파일 로드
dart_df = pd.read_csv(DART_CORP_MAP_PATH, dtype={"corp_code": str})
dart_df['corp_name'] = dart_df['corp_name'].astype(str).str.strip()

# ✅ 3. 교집합 기준 종목 추출
matched_df = dart_df[dart_df['corp_name'].isin(unique_names)].copy()
matched_df = matched_df.reset_index(drop=True)

# ✅ 4. 저장 및 결과 출력
matched_df.to_csv(SAVE_MATCHED_PATH, index=False, encoding='utf-8-sig')
print(f"✅ DART에 등록된 종목 수: {len(matched_df)}개")
print(f"📁 저장 완료 → {SAVE_MATCHED_PATH}")

✅ DART에 등록된 종목 수: 2400개
📁 저장 완료 → /Users/gun/Desktop/경종설/국내/DART/보유종목_dart코드매핑.csv


In [57]:
import pandas as pd
import os

# 연도별 파일 및 기준월 매핑
year_quarter_map = {
    "2021": {
        "2021-1.csv": ["2021-01-01", "2021-02-01", "2021-03-01"],
        "2021-2.csv": ["2021-04-01", "2021-05-01", "2021-06-01"],
        "2021-3.csv": ["2021-07-01", "2021-08-01", "2021-09-01"],
        "2021-4.csv": ["2021-10-01", "2021-11-01", "2021-12-01"],
    },
    "2022": {
        "2022-1.csv": ["2022-01-01", "2022-02-01", "2022-03-01"],
        "2022-2.csv": ["2022-04-01", "2022-05-01", "2022-06-01"],
        "2022-3.csv": ["2022-07-01", "2022-08-01", "2022-09-01"],
        "2022-4.csv": ["2022-10-01", "2022-11-01", "2022-12-01"],
    },
    "2023": {
        "2023-1.csv": ["2023-01-01", "2023-02-01", "2023-03-01"],
        "2023-2.csv": ["2023-04-01", "2023-05-01", "2023-06-01"],
        "2023-3.csv": ["2023-07-01", "2023-08-01", "2023-09-01"],
        "2023-4.csv": ["2023-10-01", "2023-11-01", "2023-12-01"],
    },
    "2024": {
        "2024-1.csv": ["2024-01-01", "2024-02-01", "2024-03-01"],
        "2024-2.csv": ["2024-04-01", "2024-05-01", "2024-06-01"],
        "2024-3.csv": ["2024-07-01", "2024-08-01", "2024-09-01"],
        "2024-4.csv": ["2024-10-01", "2024-11-01", "2024-12-01"],
    },
}

# base 경로 (로컬 환경에 맞게 수정)
base_path = "/Users/gun/Desktop/경종설/국내/DART/raw_data/"
df_all = []

for year, quarter_map in year_quarter_map.items():
    for file_name, month_list in quarter_map.items():
        file_path = os.path.join(base_path, file_name)
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"❌ {file_name} 읽기 실패:", e)
            continue

        for month in month_list:
            df_copy = df.copy()
            df_copy.insert(0, "기준월", month)
            df_all.append(df_copy)

# 병합 및 기준월 정렬
df_merged = pd.concat(df_all, ignore_index=True)
df_merged["기준월"] = pd.to_datetime(df_merged["기준월"])
df_sorted = df_merged.sort_values(by=["회사명", "기준월"]).reset_index(drop=True)

# 결측치가 하나라도 있는 기업 전체 제거
df_cleaned = df_sorted.groupby("회사명").filter(lambda x: not x.isnull().any().any()).reset_index(drop=True)

# 저장
output_path = os.path.join(base_path, "2021_2024_원본_기준월통합_결측제거.csv")
df_cleaned.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 기준월 통합 + 결측치 기업 제거 완료:", output_path)

✅ 기준월 통합 + 결측치 기업 제거 완료: /Users/gun/Desktop/경종설/국내/DART/raw_data/2021_2024_원본_기준월통합_결측제거.csv


In [58]:
import pandas as pd

# 1. 파일 경로 (로컬에 맞게 수정하세요)
file_path = "/Users/gun/Desktop/경종설/국내/DART/2021_2024_원본_기준월통합_결측제거.csv"

# 2. CSV 파일 불러오기
df = pd.read_csv(file_path)

# 3. 필요한 컬럼 숫자형으로 변환
numeric_cols = ['자본총계', '부채총계', '당기순이익', '영업이익', '매출액']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(",", "").str.strip(), errors='coerce')

# 4. 재무 비율 계산
df['ROE'] = (df['당기순이익'] / df['자본총계']) * 100
df['영업이익률'] = (df['영업이익'] / df['매출액']) * 100
df['부채비율'] = (df['부채총계'] / df['자본총계']) * 100

# 5. 필요한 컬럼만 추출
df_finance = df[['기준월', '회사명', 'ROE', '영업이익률', '부채비율']].copy()

# 6. 저장
output_path = "/Users/gun/Desktop/경종설/국내/DART/2021_2024_재무제표.csv"
df_finance.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 저장 완료:", output_path)

✅ 저장 완료: /Users/gun/Desktop/경종설/국내/DART/2021_2024_재무제표.csv


In [59]:
import pandas as pd

# 1. 파일 경로 (로컬에 맞게 수정하세요)
file_path = "/Users/gun/Desktop/경종설/국내/DART/2021_2024_재무제표.csv"

# 2. 파일 불러오기
df = pd.read_csv(file_path)

# 3. 결측치가 하나라도 있는 행 제거
df_cleaned = df.dropna(subset=['ROE', '영업이익률', '부채비율']).reset_index(drop=True)

# 4. 덮어쓰기 저장
df_cleaned.to_csv(file_path, index=False, encoding='utf-8-sig')

print(f"✅ 결측치 제거 후 저장 완료: {file_path}")
print(f"🧾 최종 행 수: {len(df_cleaned)}")

✅ 결측치 제거 후 저장 완료: /Users/gun/Desktop/경종설/국내/DART/2021_2024_재무제표.csv
🧾 최종 행 수: 112593


In [76]:
import pandas as pd

# 경로 설정
path_fin = '/Users/gun/Desktop/경종설/국내/2021_2024_원본_기준월통합_결측제거.csv'
path_mkt = '/Users/gun/Desktop/경종설/국내/2020-2024시장지표.csv'
path_output = '/Users/gun/Desktop/경종설/국내/2021_2024_재무시장_병합_및지표계산.csv'

# 파일 로드
df_fin = pd.read_csv(path_fin)
df_mkt = pd.read_csv(path_mkt)

# 기준월, 회사명 정렬 및 공백 제거
df_fin['기준월'] = df_fin['기준월'].astype(str).str.strip()
df_fin['회사명'] = df_fin['회사명'].astype(str).str.strip()
df_mkt['기준월'] = df_mkt['기준월'].astype(str).str.strip()
df_mkt['회사명'] = df_mkt['회사명'].astype(str).str.strip()

# 병합
df_merge = pd.merge(df_fin, df_mkt, on=['기준월', '회사명'], how='inner')

# 저장
df_merge.to_csv(path_output, index=False)
print(f"✅ 저장 완료: {path_output}")

✅ 저장 완료: /Users/gun/Desktop/경종설/국내/2021_2024_재무시장_병합_및지표계산.csv


In [79]:
import pandas as pd

# 📂 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_재무시장_병합_및지표계산.csv'
output_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종.csv'

# 📌 데이터 불러오기
df = pd.read_csv(file_path)

# 📌 지표 계산에 필요한 컬럼 리스트
numeric_cols = [
    '시가총액', '당기순이익', '자본총계', '매출액', '영업이익',
    '발행주식수', '부채총계'
]

# 📌 문자열로 된 숫자를 float으로 변환 (콤마 제거 등 포함)
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

# ✅ 재무 비율 계산
df['PER'] = df['시가총액'] / df['당기순이익']
df['PBR'] = df['시가총액'] / df['자본총계']
df['PSR'] = df['시가총액'] / df['매출액']
df['ROE'] = df['당기순이익'] / df['자본총계']
df['영업이익률'] = df['영업이익'] / df['매출액']
df['EPS'] = df['당기순이익'] / df['발행주식수']
df['부채비율'] = df['부채총계'] / df['자본총계']

# 🧹 무한대, NaN 처리
df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# 📁 저장
df.to_csv(output_path, index=False)
print(f"✅ 계산 완료 및 저장: {output_path}")

✅ 계산 완료 및 저장: /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종.csv


In [84]:
import pandas as pd

# ✅ 파일 경로 설정
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종.csv'
output_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_결측치제거완료.csv'

# ✅ 데이터 불러오기
df = pd.read_csv(file_path)

# ✅ 결측치 포함 여부를 기업 단위로 확인
기업별_결측치 = df.groupby('회사명').apply(lambda x: x.isnull().any().any())
결측치_있는_기업 = 기업별_결측치[기업별_결측치].index.tolist()

# ✅ 결측치 포함 기업 제거
df_filtered = df[~df['회사명'].isin(결측치_있는_기업)]

# ✅ 결과 저장
df_filtered.to_csv(output_path, index=False)

# ✅ 통계 출력
print(f"총 제거된 기업 수: {len(결측치_있는_기업)}개")
print(f"남은 기업 수: {df_filtered['회사명'].nunique()}개")
print(f"결과 저장 완료: {output_path}")

/var/folders/wz/q6p9zl7j4tj73mfvrhf322k80000gn/T/ipykernel_6564/3527935633.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  기업별_결측치 = df.groupby('회사명').apply(lambda x: x.isnull().any().any())


총 제거된 기업 수: 519개
남은 기업 수: 1789개
결과 저장 완료: /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_결측치제거완료.csv


In [94]:
import pandas as pd

# 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종.csv'
df = pd.read_csv(file_path)

# 회사명별 기준월 개수 세기
company_counts = df['회사명'].value_counts()

# 12개월 미만 데이터 가진 회사명 추출
incomplete_companies = company_counts[company_counts < 12].index.tolist()

# 결과 출력
print("12개월 미만 데이터 보유 기업 수:", len(incomplete_companies))
print("기업명 목록:")
for company in incomplete_companies:
    print(company)

12개월 미만 데이터 보유 기업 수: 5
기업명 목록:
에이텀
다원넥스뷰
스틱인베스트먼트
에이치엔에스하이텍
듀켐바이오


In [95]:
import pandas as pd

# ✅ 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종.csv'
df = pd.read_csv(file_path)

# ✅ 회사명별 기준월 개수 세기
company_counts = df['회사명'].value_counts()

# ✅ 12개월 미만 데이터 가진 회사명 추출
incomplete_companies = company_counts[company_counts < 12].index.tolist()

# ✅ 해당 기업 제거
df_filtered = df[~df['회사명'].isin(incomplete_companies)].reset_index(drop=True)

# ✅ 결과 저장
output_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업만.csv'
df_filtered.to_csv(output_path, index=False)

# ✅ 요약 출력
print("제거된 기업 수:", len(incomplete_companies))
print("남은 기업 수:", df_filtered['회사명'].nunique())
print("남은 전체 행 수:", len(df_filtered))

제거된 기업 수: 5
남은 기업 수: 2303
남은 전체 행 수: 103548


In [96]:
import pandas as pd

# 📌 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업만.csv'
df = pd.read_csv(file_path)

# 📌 결측치 개수 확인
missing_count_per_column = df.isnull().sum()

# 📌 전체 결측치 개수
total_missing = df.isnull().sum().sum()

# 📌 결측치가 있는 행 수
rows_with_nan = df.isnull().any(axis=1).sum()

print("📌 컬럼별 결측치 개수:\n", missing_count_per_column)
print("\n📌 전체 결측치 총합:", total_missing)
print("📌 결측치 포함된 행 수:", rows_with_nan)

📌 컬럼별 결측치 개수:
 기준월            0
회사명            0
자산총계           0
자본총계         771
부채총계         594
자본금            0
매출액         3735
영업이익         765
당기순이익        864
종목코드           0
산업군            0
1개월수익률       165
3개월수익률       165
6개월수익률       165
12개월수익률      165
3개월변동성       193
12개월변동성      193
평균거래량증가율    2296
거래회전율       2788
시가총액         165
발행주식수          0
PER         1032
PBR          939
PSR         4159
ROE         1074
영업이익률       4197
EPS          864
부채비율         792
dtype: int64

📌 전체 결측치 총합: 26081
📌 결측치 포함된 행 수: 7380


In [97]:
# 📌 결측치 제거
df_cleaned = df.dropna().reset_index(drop=True)

# 📌 저장
output_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업_결측치제거.csv'
df_cleaned.to_csv(output_path, index=False)

# 📌 요약 출력
print("✅ 결측치 제거 완료")
print("✅ 남은 행 수:", len(df_cleaned))
print("✅ 남은 기업 수:", df_cleaned['회사명'].nunique())

✅ 결측치 제거 완료
✅ 남은 행 수: 96168
✅ 남은 기업 수: 2254


In [98]:
import pandas as pd

# 📌 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업_결측치제거.csv'
df = pd.read_csv(file_path)

# 📌 기준월에서 연도 추출
df['연도'] = df['기준월'].astype(str).str[:4]

# 📌 연도별 고유 기업 수 계산
company_count_by_year = df.groupby('연도')['회사명'].nunique()

# 📌 출력
print("✅ 연도별 고유 기업 수:")
print(company_count_by_year)

✅ 연도별 고유 기업 수:
연도
2021    1914
2022    2008
2023    2111
2024    2209
Name: 회사명, dtype: int64


In [109]:
import pandas as pd

# CSV 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업만.csv'
df = pd.read_csv(file_path)

# 기준월에서 연도 추출
df['연도'] = df['기준월'].astype(str).str[:4]

# 연도별 고유 기업 수 세기
yearly_company_counts = df.groupby('연도')['회사명'].nunique()

# 결과 출력
print("연도별 고유 기업 수:")
print(yearly_company_counts)

연도별 고유 기업 수:
연도
2021    2031
2022    2125
2023    2205
2024    2303
Name: 회사명, dtype: int64


In [110]:
import pandas as pd

# 파일 로드
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업만.csv'
df = pd.read_csv(file_path)

# 연도별 기업별 기준월 개수 세기
df['연도'] = df['기준월'].astype(str).str[:4]
company_year_counts = df.groupby(['회사명', '연도']).size()

# 기준월이 12개가 아닌 회사-연도 조합 확인
not_full = company_year_counts[company_year_counts != 12]

# 결과 출력
print("✅ 12개월 미만인 기업-연도 조합 수:", len(not_full))

✅ 12개월 미만인 기업-연도 조합 수: 72


In [111]:
import pandas as pd

# 1. 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_12개월기업만.csv'
df = pd.read_csv(file_path)

# 2. 연도 컬럼 생성
df['연도'] = df['기준월'].astype(str).str[:4]

# 3. 회사명 + 연도 조합 중 기준월 개수 세기
group_counts = df.groupby(['회사명', '연도']).size().reset_index(name='count')

# 4. 기준월이 12개인 조합만 필터링
valid_combinations = group_counts[group_counts['count'] == 12][['회사명', '연도']]

# 5. 원본 데이터와 inner join으로 유효한 조합만 남기기
df_filtered = df.merge(valid_combinations, on=['회사명', '연도'], how='inner')

# 6. 저장
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_완전정상기업.csv'
df_filtered.to_csv(save_path, index=False)

print("✅ 저장 완료:", save_path)
print("📊 최종 기업 수:", df_filtered['회사명'].nunique())

✅ 저장 완료: /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_완전정상기업.csv
📊 최종 기업 수: 2301


In [112]:
import pandas as pd

# 파일 로드
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱최종_완전정상기업.csv'
df = pd.read_csv(file_path)

# 연도별 기업별 기준월 개수 세기
df['연도'] = df['기준월'].astype(str).str[:4]
company_year_counts = df.groupby(['회사명', '연도']).size()

# 기준월이 12개가 아닌 회사-연도 조합 확인
not_full = company_year_counts[company_year_counts != 12]

# 결과 출력
print("✅ 12개월 미만인 기업-연도 조합 수:", len(not_full))

✅ 12개월 미만인 기업-연도 조합 수: 0


In [115]:
import pandas as pd

# ✅ CSV 파일 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_재무.csv'
df = pd.read_csv(file_path)

# ✅ '기준월' 컬럼에서 연도 추출
df['연도'] = df['기준월'].astype(str).str[:4]

# ✅ 연도별 기업 수 확인 (회사명 중복 제거 후 개수 세기)
result = df.groupby('연도')['회사명'].nunique()

# ✅ 출력
print("📌 연도별 기업 수:")
print(result)

📌 연도별 기업 수:
연도
2021    1997
2022    1997
2023    1997
2024    1997
Name: 회사명, dtype: int64


In [133]:
import pandas as pd

# ✅ 원본 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_재무.csv'

# ✅ 저장할 새 파일 경로
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_표준계정삭제.csv'

# ✅ CSV 파일 불러오기
df = pd.read_csv(file_path)

# ✅ "표준계정코드 미사용" 또는 "조회결과없음"이 포함된 행 탐색
mask = df.apply(lambda row: row.astype(str).str.contains('표준계정코드 미사용|조회결과없음').any(), axis=1)

# ✅ 해당 행 제거
df_cleaned = df[~mask].copy()

# ✅ 새로운 파일로 저장
df_cleaned.to_csv(save_path, index=False)

print(f"✅ 저장 완료: {save_path}")

✅ 저장 완료: /Users/gun/Desktop/경종설/국내/2021_2024_표준계정삭제.csv


In [137]:
import pandas as pd

# ✅ 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_표준계정삭제.csv'

# ✅ CSV 파일 불러오기
df = pd.read_csv(file_path)

# ✅ 행 기준으로 결측치가 하나라도 있는 경우 필터링
companies_with_nan = df[df.isna().any(axis=1)]['회사명'].unique()

# ✅ 결과 출력
print(f"❗결측치가 **하나라도 존재하는 기업 수**: {len(companies_with_nan)}개")

❗결측치가 **하나라도 존재하는 기업 수**: 390개


In [138]:
import pandas as pd

# ✅ 경로 확인 후 수정
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_표준계정삭제.csv'

# ✅ 파일 불러오기
df = pd.read_csv(file_path)

# ✅ 결측치가 하나라도 있는 기업 추출
companies_with_nan = df[df.isna().any(axis=1)]['회사명'].unique()

# ✅ 해당 기업 제거
df_cleaned = df[~df['회사명'].isin(companies_with_nan)].copy()

# ✅ 저장
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_삭제삭제.csv'
df_cleaned.to_csv(save_path, index=False)

In [142]:
import pandas as pd

# ✅ 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_삭제삭제.csv'

# ✅ 데이터 불러오기
df = pd.read_csv(file_path)

# ✅ 1. 전체 기업 수 (중복 제거)
total_companies = df['회사명'].nunique()
print(f"✅ 전체 기업 수: {total_companies}개")

# ✅ 2. 연도별 기업 수
companies_per_year = df.groupby('연도')['회사명'].nunique()
print("\n✅ 연도별 기업 수:")
print(companies_per_year)

# ✅ 3. 전체 결측치 개수
total_missing = df.isna().sum().sum()
print(f"\n✅ 전체 결측치 개수: {total_missing}개")

✅ 전체 기업 수: 1589개

✅ 연도별 기업 수:
연도
2021    1566
2022    1569
2023    1577
2024    1588
Name: 회사명, dtype: int64

✅ 전체 결측치 개수: 0개


In [143]:
# 기업별 연도 카운트
year_count_per_company = df.groupby('회사명')['연도'].nunique()

# 4년 모두 존재하지 않는 기업 필터링
not_4_years = year_count_per_company[year_count_per_company < 4]
print(f"❌ 4년 모두 존재하지 않는 기업 수: {len(not_4_years)}개")
print("\n❌ 연도 누락된 대표 기업:")
print(not_4_years.sort_values())

❌ 4년 모두 존재하지 않는 기업 수: 28개

❌ 연도 누락된 대표 기업:
회사명
AP헬스케어      1
에프엔에스테크     1
아시아경제       1
한독          1
이연제약        1
무림SP        1
리드코프        1
무림P&P       1
국순당         1
유유제약        1
이화산업        2
월덱스         2
크린앤사이언스     2
미스토홀딩스      2
기아          2
이마트         2
NEW         2
롯데하이마트      2
한국정보공학      3
유진로봇        3
셀트리온제약      3
아이티센씨티에스    3
삼진제약        3
두산밥캣        3
대성파인텍       3
농우바이오       3
에코프로비엠      3
핸디소프트       3
Name: 연도, dtype: int64


In [153]:
import pandas as pd

# ✅ 데이터 불러오기
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_삭제삭제.csv'
df = pd.read_csv(file_path)

# ✅ 기업별 연도 수 확인
year_count_per_company = df.groupby('회사명')['연도'].nunique()

# ✅ 4년 모두 존재하는 기업만 필터링
valid_companies = year_count_per_company[year_count_per_company == 4].index
df_valid = df[df['회사명'].isin(valid_companies)].copy()

# ✅ 저장
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱.csv'
df_valid.to_csv(save_path, index=False)
print(f"✅ 저장 완료: {save_path}")

✅ 저장 완료: /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱.csv


In [155]:
import pandas as pd

# ✅ 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱.csv'

# ✅ CSV 파일 불러오기
df = pd.read_csv(file_path)

# ✅ 회사별 등장 횟수 계산
company_counts = df['회사명'].value_counts()

# ✅ 48번 등장한 회사만 필터링
valid_companies = company_counts[company_counts == 48].index
df_filtered = df[df['회사명'].isin(valid_companies)].copy()

# ✅ 덮어쓰기 저장
df_filtered.to_csv(file_path, index=False)
print(f"✅ 48번 등장한 {len(valid_companies)}개 기업만 남기고 덮어쓰기 저장 완료")

✅ 48번 등장한 1531개 기업만 남기고 덮어쓰기 저장 완료


In [162]:
import pandas as pd
import numpy as np

# ✅ 경로 설정
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱.csv'
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환.csv'

# ✅ CSV 로드
df = pd.read_csv(file_path)

# ✅ 변환 제외 컬럼
exclude_cols = ['기준월', '회사명', '산업군']

# ✅ 숫자형 변환 처리 (쉼표 제거 + 지수포기 포함)
for col in df.columns:
    if col not in exclude_cols and df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# ✅ 숫자형 포맷 (정수는 쉼표, 소수는 원래 소수 그대로 출력)
for col in df.columns:
    if col not in exclude_cols and pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].apply(
            lambda x: f"{int(x):,}" if pd.notnull(x) and float(x).is_integer()
            else f"{x}" if pd.notnull(x)
            else ""
        )

# ✅ 저장 (인코딩: utf-8-sig → 한글 깨짐 방지)
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"✅ 변환 및 저장 완료 (utf-8-sig 인코딩): {save_path}")

✅ 변환 및 저장 완료 (utf-8-sig 인코딩): /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환.csv


In [179]:
import pandas as pd

# 파일 경로
file_path = '/Users/gun/Desktop/경종설/국내/stock_0.3ver.csv'

# CSV 불러오기
df = pd.read_csv(file_path)

# 1. 전체 기업 수
num_companies = df['회사명'].nunique()

# 2. 기업별 데이터 개수 확인
company_counts = df['회사명'].value_counts()
not_48 = company_counts[company_counts != 48]

# 3. 총 행 수와 기대 행 수 비교
expected_rows = num_companies * 48
actual_rows = len(df)
all_rows_ok = (expected_rows == actual_rows)

# 출력
print(f"✅ 전체 기업 수: {num_companies}")
print(f"✅ 48번이 아닌 기업 수: {len(not_48)}")
if len(not_48) > 0:
    print("❌ 48번이 아닌 기업 목록:")
    print(not_48)
print(f"✅ 총 행 수: {actual_rows} / 기대 행 수: {expected_rows}")
print("✅ 전체 행 수 일치 여부:", "일치" if all_rows_ok else "❌ 불일치")

✅ 전체 기업 수: 1529
✅ 48번이 아닌 기업 수: 0
✅ 총 행 수: 73392 / 기대 행 수: 73392
✅ 전체 행 수 일치 여부: 일치


In [181]:
df.head()

,기준월,회사명,자산총계,자본총계,부채총계,자본금,매출액,영업이익,당기순이익,종목코드,...,EPS,부채비율,momentum_diff,momentum_accel,trend_positive_count,return_vol_ratio,volatility_spread,roe_per_ratio,momentum_trend_slope,monthly_return_cv
0,2021-01-01,AJ네트웍스,1218314937498,296631054838,921683882660,46822295000,127902006965,6566580373,2210599293,95570,...,48.850045,3.107173,0.508327,0.387435,3,12.500063,0.002167,0.000079,-0.021128,1.331939
1,2021-02-01,AJ네트웍스,1218314937498,296631054838,921683882660,46822295000,127902006965,6566580373,2210599293,95570,...,48.850045,3.107173,0.293916,0.242159,2,3.658349,0.004446,0.000090,-0.010816,10.011748
2,2021-03-01,AJ네트웍스,1218314937498,296631054838,921683882660,46822295000,127902006965,6566580373,2210599293,95570,...,48.850045,3.107173,-0.088904,-0.092539,1,-5.033027,0.014573,0.000092,0.005066,5.535426
3,2021-04-01,AJ네트웍스,1164109998733,313431955665,850678043068,46822295000,248112754573,23259137341,19378839665,95570,...,428.235540,2.714076,-0.619969,-0.162645,4,0.051567,0.006928,0.005731,0.050858,0.908207
4,2021-05-01,AJ네트웍스,1164109998733,313431955665,850678043068,46822295000,248112754573,23259137341,19378839665,95570,...,428.235540,2.714076,-0.112384,0.172774,4,22.619561,0.009339,0.004762,0.023027,0.369591


In [164]:
import pandas as pd

# ✅ 경로 설정
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환.csv'
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환_완성.csv'

# ✅ 파일 불러오기
df = pd.read_csv(file_path)

# ✅ 변환 제외 컬럼
exclude_cols = ['기준월', '회사명', '산업군']

# ✅ 숫자형 변환 (쉼표 제거, 지수표기 포함)
for col in df.columns:
    if col not in exclude_cols and df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.replace(',', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# ✅ 저장 (UTF-8 인코딩)
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"✅ 숫자형 변환 완료 및 저장: {save_path}")

✅ 숫자형 변환 완료 및 저장: /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환_완성.csv


In [165]:
import pandas as pd

# ✅ 경로 설정
file_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환_완성.csv'
save_path = '/Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환_최종.csv'

# ✅ CSV 로드
df = pd.read_csv(file_path)

# ✅ '연도' 열 삭제 (존재할 경우만)
if '연도' in df.columns:
    df.drop(columns='연도', inplace=True)

# ✅ 숫자형 컬럼 유지한 채 저장 (쉼표 없이)
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"✅ 최종 저장 완료: {save_path}")

✅ 최종 저장 완료: /Users/gun/Desktop/경종설/국내/2021_2024_리밸런싱_형변환_최종.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# ✅ 파일 경로
file_path = "/Users/gun/Desktop/경종설/국내/시장+재무/2021_2024_리밸런싱최종.csv"
save_path = "/Users/gun/Desktop/경종설/국내/quant_features.csv"

# ✅ 데이터 불러오기
df = pd.read_csv(file_path)

# ✅ 수익률 컬럼
returns_cols = ['1개월수익률', '3개월수익률', '6개월수익률', '12개월수익률']

# ✅ 피처 계산
df['momentum_diff'] = df['3개월수익률'] - df['12개월수익률']
df['momentum_accel'] = df['3개월수익률'] - df['1개월수익률']
df['trend_positive_count'] = df[returns_cols].gt(0).sum(axis=1)
df['return_vol_ratio'] = df['3개월수익률'] / df['3개월변동성'].replace(0, np.nan)
df['volatility_spread'] = df['12개월변동성'] - df['3개월변동성']
df['roe_per_ratio'] = df['ROE'] / df['PER'].replace(0, np.nan)

# ✅ momentum_trend_slope (선형회귀 기울기)
def calc_slope(row):
    y = row[returns_cols].values
    if np.any(pd.isna(y)):
        return np.nan
    x = np.array([1, 3, 6, 12]).reshape(-1, 1)
    model = LinearRegression().fit(x, y)
    return model.coef_[0]

df['momentum_trend_slope'] = df[returns_cols].apply(calc_slope, axis=1)

# ✅ monthly_return_cv (수익률 계열의 변동계수)
df['monthly_return_cv'] = df[returns_cols].std(axis=1) / df[returns_cols].mean(axis=1).replace(0, np.nan)

# ✅ 시가총액 증가 여부
df['market_cap_growth_flag'] = df.groupby('회사명')['시가총액'].diff().gt(0).astype(int)

# ✅ 저장
df.to_csv(save_path, index=False, encoding='utf-8-sig') 
print(f"✅ 최종 피처셋 저장 완료: {file_path}")

✅ 최종 피처셋 저장 완료: /Users/gun/Desktop/경종설/국내/quant_features.csv


In [178]:
import pandas as pd

# ✅ 기존 파일 경로
file_path = "/Users/gun/Desktop/경종설/국내/quant_features.csv"
df = pd.read_csv(file_path)
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"✅ 찐막 파일 저장 완료: {file_path}")

✅ 찐막 파일 저장 완료: /Users/gun/Desktop/경종설/국내/quant_features.csv
